In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module infowindow
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols
loading module statsplotwindow
loading module foldsplit
loading module histogram
loading module save_resampled
loading module simple_controls
loading module valuesplotwindow
loading module gradientpalette
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
#embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/ScCal-SLi_right/09-28-12_1/imagen_random_embeddings/full_embeddings.csv"
embeddings_file = "/neurospin/dico/jlaval/Output/imagen_right/22-26-14_254/imagen_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/patterns/interruptions.csv"
#participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"

In [4]:
#label = "C0_L"
label = "Right_Interrup_RS_OTS"
#label = "Right_Interrup_RS_CS"

In [5]:
side = "R" # "R" or "L"
#region = "Sc.Cal.-S.Li."# "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
region = "deMatos."
database='imagen'

# Building predictors

In [6]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [7]:
participants.head()

,Right_Interrup_RS_CS
Subject,
1274,1
22453,1
75717,0
106601,0
106871,0


In [8]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([1, 0])

In [9]:
len(participants)

2005

In [10]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [11]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
ID,,,,,,,,,,,,,,,,,,,,,
1274,2.885589,-3.127934,-0.114491,-3.495417,7.985891,-0.925394,1.638067,0.629670,1.242110,4.047585,...,1.582842,-3.789710,0.459150,-2.619864,1.209171,-2.816818,2.375491,4.838028,1.034805,-2.420225
22453,0.779331,6.862106,-0.233531,-3.541402,2.700162,1.808862,-3.083261,-1.567097,0.099141,0.325716,...,-7.145650,-4.192442,0.746335,0.588816,4.803193,4.128208,-0.755767,2.041151,2.063539,-1.259303
75717,6.573874,2.941595,1.542222,1.137285,-0.866760,3.628838,-1.252701,0.303853,-0.503919,-1.404402,...,-7.552640,-0.440594,-1.601537,-2.747158,-3.890281,-3.527558,1.639064,4.347117,-1.674291,4.271338
106601,-0.156071,1.388628,-2.208508,1.025495,1.454451,-5.355864,1.165401,-3.978768,-1.079862,-2.573967,...,-5.994969,4.688922,-3.911742,-2.047635,1.709245,6.624426,1.875559,3.701482,0.882760,2.328465
106871,-0.277742,1.938218,2.189120,-1.294978,0.715511,2.619005,1.154050,0.923240,0.925369,-4.977792,...,-2.790260,2.707063,-0.500233,-0.904309,-2.951421,2.011630,0.077940,-1.631355,0.409057,4.704006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873252,2.864254,1.747673,0.483137,-3.300758,6.663378,2.447789,-1.451873,-5.953968,0.320539,2.561995,...,-1.007424,-5.007984,0.581549,-2.856060,3.807414,2.630482,0.590811,4.538923,2.869514,-1.572596
99875982,1.470226,6.191041,2.506579,0.246011,-3.704151,3.077160,1.090319,2.040973,-4.881302,0.426182,...,-3.737983,-0.934120,-7.434391,1.467696,-8.028265,0.038181,-2.166672,-0.557899,-0.327631,5.229538
99888850,-1.309783,4.155358,-0.880598,-1.324898,4.115377,7.620597,1.431535,0.674450,0.706580,2.991778,...,-4.722950,2.289989,-0.182332,-0.074694,0.061270,-0.191474,-1.901747,2.373237,3.070154,3.698780


In [12]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

108.68210105514382

In [13]:
## here use directly the classif scores
df = pd.read_csv(f'/neurospin/dico/jlaval/Output/imagen_right/22-26-14_254/imagen_random_embeddings/{label}/full_predicted_probas.csv')
df_ = pd.DataFrame({label: df['label'], 'predicted': df['mean_pred']})
df_.index=df['Subject']
df = df_.copy()

In [14]:
df.mean()

Right_Interrup_RS_CS    0.358321
predicted               0.358243
dtype: float64

In [15]:
df.head()

,Right_Interrup_RS_CS,predicted
Subject,,
1274,1,0.762903
22453,1,0.041471
75717,0,0.307026
106601,0,0.075840
106871,0,0.006037


### Function definitions

In [16]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() == 'imagen':
            subject_id_list = [str(num).zfill(12) for num in subject_id_list]
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [19]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=50, nb_columns=5, nb_lines=3):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [20]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x7775cc9ad480> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cc9d79a0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cc9d7640> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cc9af130> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cc9aeb00> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca0caf0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca0c5e0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca0d750> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca137f0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca19e10> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca205e0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca25fc0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca25e10> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca26cb0> (40, 62, 42, 1)
<soma.aims.Volume_DOUBLE object at 0x7775cca2ff40> (40, 62, 42

In [18]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [19]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x7fcab0915ea0>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x7fca85764940>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca74394f70>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x7fcab0993c70>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x7fca74362050>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca74362290>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x7fca743620e0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x7fca743b0a60>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x7fca743b11b0>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
